In [1]:
import pandas as pd
import quandl as qd
import numpy as np
import math
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import preprocessing, cross_validation
from sklearn import svm
from sklearn.preprocessing import normalize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import PyQt5
from sklearn.neural_network import MLPRegressor
import tensorflow
import keras.utils
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout

/home/nishchal/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
df = qd.get("NSE/POWERGRID", authtoken="gmWv5h5b4KEUxUbiSneX")

In [3]:
df.head()

,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
Date,,,,,,,
2007-10-05,89.8,109.50,83.15,100.20,100.60,481058927.0,493056.07
2007-10-08,104.0,104.00,94.85,95.30,95.45,71252858.0,69916.16
2007-10-09,95.5,102.85,90.35,102.10,101.95,65658228.0,64717.84
2007-10-10,104.6,105.00,102.00,102.65,102.90,38211410.0,39608.87
2007-10-11,120.0,120.00,102.00,114.85,114.40,59805552.0,66252.35


In [4]:
df.dropna(inplace=True)

In [5]:
df['Date'] = np.array(df.index.to_pydatetime(), dtype=np.datetime64)

In [29]:
def RSI(dataframe, period):
    delta = dataframe.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
    d = d.drop(d.index[:(period-1)])
    rs = pd.stats.moments.ewma(u, com=period-1, adjust=False) / \
         pd.stats.moments.ewma(d, com=period-1, adjust=False)
    return 100 - 100 / (1 + rs)
df_temp = df[['Open', 'High', 'Low', 'Close', 'Total Trade Quantity']]
df_temp['HLP'] = (df_temp['High'] - df_temp['Close']) / df_temp['Close'] * 100.0
df_temp['Change'] = (df_temp['Close'] - df_temp['Open']) / df_temp['Open'] * 100
df_temp['EPS'] = (df['Turnover (Lacs)']*100000)/df['Total Trade Quantity']
df_temp['PE'] = df_temp['Close']/df_temp['EPS']
df_temp['EWMA12']= pd.ewma(df_temp['Close'], span=12)
df_temp['EWMA26']= pd.ewma(df_temp['Close'], span=26)
df_temp['MACD']= (df_temp['EWMA12'] - df_temp['EWMA26'])
df_temp['RSI']= RSI(df_temp['Close'],6)
def getclass(int):
    if(int < 0):
        return 0
    else:
        return 1
        
df_temp['Class']= df_temp['Change'].apply(getclass)
df_use = df_temp[['Close', 'HLP', 'Change', 'Total Trade Quantity', 'EPS', 'PE' , 'EWMA12','EWMA26','MACD', 'RSI']]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:18: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(min_periods=0,span=12,ignore_na=False,adjust=True).mean()
/usr/local/lib/python3.5/dist-packages/ipy

In [30]:
df_use.describe()

,Close,HLP,Change,Total Trade Quantity,EPS,PE,EWMA12,EWMA26,MACD,RSI
count,2652.000000,2652.000000,2652.000000,2.652000e+03,2652.000000,2652.000000,2652.000000,2652.000000,2652.000000,2646.000000
mean,128.714857,1.453158,-0.066296,5.188986e+06,128.749414,0.999799,128.526280,128.315811,0.210468,51.006775
std,37.440889,1.451119,2.008374,1.317505e+07,37.452316,0.008101,37.155621,36.803786,2.015306,16.238346
min,58.000000,0.050531,-16.287879,3.406300e+04,55.981979,0.926117,72.198764,74.659871,-9.626400,6.880188
25%,101.800000,0.584308,-0.947138,2.091749e+06,101.695723,0.996299,101.784387,101.852812,-0.884202,39.477607
50%,111.975000,1.037903,-0.086459,3.323502e+06,112.103769,0.999769,111.371237,110.912666,0.068304,50.991547
75%,143.912500,1.799181,0.805843,5.526354e+06,143.888150,1.003299,143.383955,142.850804,1.342273,61.900125
max,225.950000,19.457014,20.000000,4.810589e+08,225.185376,1.069923,221.359736,219.290654,6.310374,94.537791


In [31]:
df_use.dropna(inplace = True)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [32]:
print('We are predicting 8 days into the future')

We are predicting 8 days into the future


In [18]:
def build_model(inputs, output_size, neurons, activ_func="linear",
                dropout=0.10, loss="mae", optimizer="adam"):
    
    model = Sequential()

    model.add(LSTM(neurons, input_shape=(inputs.shape[1], inputs.shape[2])))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))

    model.compile(loss=loss, optimizer=optimizer)
    return model

In [33]:
a = int(df_use.shape[0]*0.80)
training_set = df_use[:a]
test_set = df_use[a:]
test_set.shape

(530, 10)

In [34]:
LSTM_training_inputs = []
for i in range(len(training_set)-8):
    temp_set = training_set[i:(i+8)].copy()
    
    for col in list(temp_set):
        temp_set[col] = temp_set[col]/temp_set[col].iloc[0] - 1
    
    LSTM_training_inputs.append(temp_set)

In [35]:
LSTM_test_inputs = []
for i in range(len(test_set)-8):
    temp_set = test_set[i:(i+8)].copy()
    
    for col in list(temp_set):
        temp_set[col] = temp_set[col]/temp_set[col].iloc[0] - 1
    
    LSTM_test_inputs.append(temp_set)

In [36]:
LSTM_training_inputs = [np.array(LSTM_training_input) for LSTM_training_input in LSTM_training_inputs]
LSTM_training_inputs = np.array(LSTM_training_inputs)

LSTM_test_inputs = [np.array(LSTM_test_input) for LSTM_test_input in LSTM_test_inputs]
LSTM_test_inputs = np.array(LSTM_test_inputs)

In [37]:
LSTM_training_outputs = (training_set['Close'][8:].values/training_set['Close'][:-8].values)-1

LSTM_test_outputs = (test_set['Close'][8:].values/test_set['Close'][:-8].values)-1

In [39]:
LSTM_training_inputs.shape

(2108, 8, 10)

In [ ]:
nn_model = build_model(LSTM_training_inputs, output_size=1, neurons = 32)

nn_history = nn_model.fit(LSTM_training_inputs, LSTM_training_outputs, 
                            epochs=5, batch_size=1, verbose=2, shuffle=True)

In [ ]:
plt.plot(LSTM_test_outputs, label = "actual")
plt.plot(nn_model.predict(LSTM_test_inputs), label = "predicted")
plt.legend()
plt.show()
MAE = mean_absolute_error(LSTM_test_outputs, nn_model.predict(LSTM_test_inputs))
print('The Mean Absolute Error is: {}'.format(MAE))